In [5]:
import pandas as pd 
from sklearn.model_selection import train_test_split

data = pd.read_csv('../../data/melb_data.csv')

# 독립변수 / 종속변수 분리 
y = data.Price  #종속변수

#독립변수 
melb_pred = data.drop(['Price'], axis=1) # 종속변수제외한 열 가져오기
x = melb_pred.select_dtypes(exclude=['object']) #데이터타입이 object를 제외한 열 가져오기 

#훈련용 / 검증용 데이터 분리
x_train, x_test, y_train, y_test =\
train_test_split(x, y, train_size=0.8, test_size=0.2, random_state=0)

In [6]:
#랜덤포레스트 모델 함수 

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

def score_dataset(x_train, x_test, y_train, y_test):
    #모델 구축
    model = RandomForestRegressor(n_estimators=10, random_state=0)
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    return mean_absolute_error(y_test, y_pred)
    

In [9]:
# 1. 결측치 처리 >> 삭제
#결측치가 존재하는 열 >> cols_miss
cols_miss = [col for col in x_train.columns if x_train[col].isnull().any()]

# x_train의 열중에 결측치가 존재하면 제외하겠다. 
reduced_x_train = x_train.drop(cols_miss, axis=1)
reduced_x_test = x_test.drop(cols_miss, axis=1)

print('랜덤포레스트 모델의 mae값(결측값이 있는 열 삭제): ', score_dataset(reduced_x_train, reduced_x_test, y_train, y_test))



랜덤포레스트 모델의 mae값(결측값이 있는 열 삭제):  183550.22137772635


In [12]:
# 2. 결측치 처리 >> 평균값 대치
#Simplelmputer 사용하여 결측치를 평균값으로 대체 

from sklearn.impute import SimpleImputer

si = SimpleImputer()
imputed_x_train = pd.DataFrame(si.fit_transform(x_train))
imputed_x_test = pd.DataFrame(si.transform(x_test))

imputed_x_train.columns = x_train.columns
imputed_x_test.columns = x_test.columns

print('Imputer를 이용한 MAE 값: ', score_dataset(imputed_x_train,imputed_x_test, y_train, y_test))

# Imputer를 이용한 mae값이 랜덤포레스트보다 낮아서 성능이 더 좋은것 확인 

Imputer를 이용한 MAE 값:  178166.46269899711


In [13]:
# 3. 결측치 처리 >> 값으로 대치 

x_train_plus = x_train.copy()
x_test_plus = x_test.copy()

#결측치가 있는 열을 새 열로 만들어서 담기 
for col in cols_miss:
    x_train_plus[col + '_was_missing'] = x_train_plus[col].isnull()
    x_test_plus[col + '_was_missing'] = x_test_plus[col].isnull()
    
si = SimpleImputer()
imputed_x_train_plus = pd.DataFrame(si.fit_transform(x_train_plus))
imputed_x_test_plus = pd.DataFrame(si.transform(x_test_plus))

imputed_x_train_plus.columns = x_train_plus.columns
imputed_x_test_plus.columns = x_test_plus.columns

print("확장 MAE:")
print(score_dataset(imputed_x_train_plus, imputed_x_test_plus, y_train, y_test))

# 3은 2보다 성능이 약간 나쁘게 나옴 
#삭제보다 대치가 더 성능을 발휘하는 이유
# 삭제를 하게되면 유용한 정보가 제거가 되어 데이터를 살리고 대치를 하는것이 성능이 더 좋아짐
    


확장 MAE:
178927.503183954


In [14]:
print(x_train.shape)

# 학습 데이터의 결측치 개수 
missing_val_count_by_column = (x_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

(10864, 12)
Car               49
BuildingArea    5156
YearBuilt       4307
dtype: int64
